In [ ]:
# Import Libraries
import tweepy
import pandas as pd
import numpy as np
import sys
import csv 

import requests
import os
import json
import tweepy
import pandas as pd
import numpy as np
import sys
import csv 
import matplotlib.pyplot as plt
# import textblob
# from textblob import TextBlob 
from tweepy import API
from tweepy import Cursor
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream

from tweepy import API 
from tweepy import Cursor
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
 


In [ ]:
# Fill the API Key
CONSUMER_KEY = 'EyI6UxjIgfQoABK0VOfDWEQSZ'
CONSUMER_SECRET = '3qIlP8F8GU9b5bSSWB1heaMHMyetvD53Js5SR7nPxnPk06Wst9'
ACCESS_TOKEN = '3931555760-zC4EO9fF7K05GkpTW72gRhqfkWGAGwWoLeYXCRl'
ACCESS_TOKEN_SECRET = 'H40Xosqw6hkEiqm1rK7R5dT5rX61JS8fx2SUn0D6gxfTi'

# Auth. 
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)
api

In [67]:
# Twitter Client
class TwitterClient():
    def __init__(self, twitter_user="DOHgovph"): #if None, to my own
        self.auth = TwitterAuthenticator().authenticate_twitter_app()
        self.twitter_client = API(self.auth) 

        self.twitter_user = twitter_user

    def get_twitter_client_api(self):
        return self.twitter_client

    def get_user_timeline_tweets(self, num_tweets):
        tweets = []
        for tweet in Cursor(self.twitter_client.user_timeline, id=self.twitter_user).items(num_tweets):
            tweets.append(tweet)
        return tweets

    def get_friend_list(self, num_friends):
        friend_list = []
        for friend in Cursor(self.twitter_client.friends, id=self.twitter_user).items(num_friends):
            friend_list.append(friend)
        return friend_list

    def get_home_timeline_tweets(self, num_tweets):
        home_timeline_tweets = []
        for tweet in Cursor(self.twitter_client.home_timeline_tweets, id=self.twitter_user).items(num_tweets):
            home_timeline_tweets.append(tweet)
        return home_timeline_tweets

# Twitter Authenticater
class TwitterAuthenticator():

    def authenticate_twitter_app(self):
        auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
        auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
        return auth



# # # # TWITTER STREAMER # # # #
class TwitterStreamer():
    """
    Class for streaming and processing live tweets.
    """
    def __init__(self):
        self.twitter_authenticator = TwitterAuthenticator()

    def stream_tweets(self, fetched_tweets_filename, hash_tag_list):
        # This handles Twitter authetification and the connection to Twitter Streaming API
        listener = TwitterListener(fetched_tweets_filename)
        auth = self.twitter_authenticator.authenticate_twitter_app()
        stream = Stream(auth, listener)

        # This line filter Twitter Streams to capture data by the keywords: 
        stream.filter(track=hash_tag_list)


# # # # TWITTER STREAM LISTENER # # # #
class TwitterListener(StreamListener):
    """
    This is a basic listener that just prints received tweets to stdout.
    """
    def __init__(self, fetched_tweets_filename):
        self.fetched_tweets_filename = fetched_tweets_filename

    def on_data(self, data):
        try:
            print(data)
            with open(self.fetched_tweets_filename, 'a') as tf:
                tf.write(data)
            return True
        except BaseException as e:
            print("Error on_data %s" % str(e))
        return True
          

    def on_error(self, status):
        # if status = 420:
        #     # Returning False on_data methos in case rate limit occurs
        #     return False
        print(status)

class TweetAnalyzer(): #Functionality for analyzing and categorizing content from tweets
    def tweets_to_data_frame(self, tweets):
        df = pd.DataFrame(data=[tweet.text for tweet in tweets], columns=['tweets'])

        df['id'] = np.array([tweet.id for tweet in tweets]) #add and add what u want 
        df['len'] = np.array([len(tweet.text) for tweet in tweets])
        df['date'] = np.array([tweet.created_at for tweet in tweets])
        df['source'] = np.array([tweet.source for tweet in tweets])
        df['likes'] = np.array([tweet.favorite_count for tweet in tweets])
        df['retweets'] = np.array([tweet.retweet_count for tweet in tweets])
        df['geo'] = np.array([tweet.geo for tweet in tweets])
        

        return df

 
if __name__ == '__main__':
 
    twitter_client = TwitterClient()
    tweet_analyzer = TweetAnalyzer()
    api = twitter_client.get_twitter_client_api()

    tweets = api.user_timeline(screen_name="DOHgovph", count=9000)

    df = tweet_analyzer.tweets_to_data_frame(tweets)
    
    # Get average length over all tweets
    #print(np.mean(df['len'])) # can change count to see difference

#Get the number of likes for the most liked tweet
#print(np.max(df['likes']))

#Get the number of retweets for the most retweeted tweet
print(np.max(df['retweets']))

#print(dir(tweets[0]))
#print(tweets[100].retweet_count)

# df = tweet_analyzer.tweets_to_data_frame(tweets)
    
# print(df.head(10))

104


In [47]:
user = 'DOHgovph'
tweet_id = 186347409
t = api.search(q=f'to:{user}', since_id=tweet_id,)

replies = 0
for i in range(len(t)):

    if t[i].in_reply_to_status_id == tweet_id:
        replies += 1
print(replies)

0


In [33]:
    from tweepy import OAuthHandler
    import tweepy
    pages="DOHgovph";
    for full_tweets in tweepy.Cursor(api.user_timeline, screen_name=pages).items(10):
            replyCounter=0
            try:
                #for getting comment count
                for tweet in tweepy.Cursor(api.search, q='to:' + pages, result_type='recent').items(10):
                    if hasattr(tweet, 'in_reply_to_status_id_str'):
                        if (tweet.in_reply_to_status_id_str == full_tweets.id_str):
                            replyCounter+=1

            except IOError as e:
                print("ERRROE===>",e.message)

In [49]:
tweet_id = 1280819666605072385
tweet = api.get_status(tweet_id)
print(tweet.retweet_count)
print(tweet.favorite_count)

809
1645


In [51]:
# screen name of the user
screen_name = "WHO"
  
# getting the liked statuses
favorites = api.favorites(screen_name)
  
# printing the screen names of the status posters
for status in favorites:
    print(status.user.screen_name)

DrTedros
opsoms
DrTedros
DrTedros
DrTedros
DrTedros
DrTedros
WHOAFRO
DrTedros
DrTedros
DrTedros
UNmigration
DrTedros
DrTedros
DrTedros
DrTedros
DrTedros
DrTedros
DrTedros
WHOAfghanistan


In [ ]:
from tweepy import API 
from tweepy import Cursor
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
 
import twitter_credentials

# # # # TWITTER CLIENT # # # #
class TwitterClient():
    def __init__(self, twitter_user=None):
        self.auth = TwitterAuthenticator().authenticate_twitter_app()
        self.twitter_client = API(self.auth)

        self.twitter_user = twitter_user

    def get_user_timeline_tweets(self, num_tweets):
        tweets = []
        for tweet in Cursor(self.twitter_client.user_timeline, id=self.twitter_user).items(num_tweets):
            tweets.append(tweet)
        return tweets

    def get_friend_list(self, num_friends):
        friend_list = []
        for friend in Cursor(self.twitter_client.friends, id=self.twitter_user).items(num_friends):
            friend_list.append(friend)
        return friend_list

    def get_home_timeline_tweets(self, num_tweets):
        home_timeline_tweets = []
        for tweet in Cursor(self.twitter_client.home_timeline, id=self.twitter_user).items(num_tweets):
            home_timeline_tweets.append(tweet)
        return home_timeline_tweets


# # # # TWITTER AUTHENTICATER # # # #
class TwitterAuthenticator():

    def authenticate_twitter_app(self):
        auth = OAuthHandler(twitter_credentials.CONSUMER_KEY, twitter_credentials.CONSUMER_SECRET)
        auth.set_access_token(twitter_credentials.ACCESS_TOKEN, twitter_credentials.ACCESS_TOKEN_SECRET)
        return auth

# # # # TWITTER STREAMER # # # #
class TwitterStreamer():
    """
    Class for streaming and processing live tweets.
    """
    def __init__(self):
        self.twitter_autenticator = TwitterAuthenticator()    

    def stream_tweets(self, fetched_tweets_filename, hash_tag_list):
        # This handles Twitter authetification and the connection to Twitter Streaming API
        listener = TwitterListener(fetched_tweets_filename)
        auth = self.twitter_autenticator.authenticate_twitter_app() 
        stream = Stream(auth, listener)

        # This line filter Twitter Streams to capture data by the keywords: 
        stream.filter(track=hash_tag_list)


# # # # TWITTER STREAM LISTENER # # # #
class TwitterListener(StreamListener):
    """
    This is a basic listener that just prints received tweets to stdout.
    """
    def __init__(self, fetched_tweets_filename):
        self.fetched_tweets_filename = fetched_tweets_filename

    def on_data(self, data):
        try:
            print(data)
            with open(self.fetched_tweets_filename, 'a') as tf:
                tf.write(data)
            return True
        except BaseException as e:
            print("Error on_data %s" % str(e))
        return True
          
    def on_error(self, status):
        if status == 420:
            # Returning False on_data method in case rate limit occurs.
            return False
        print(status)

 
if __name__ == '__main__':
 
    # Authenticate using config.py and connect to Twitter Streaming API.
    hash_tag_list = ["donal trump", "hillary clinton", "barack obama", "bernie sanders"]
    fetched_tweets_filename = "tweets.txt"

    twitter_client = TwitterClient('pycon')
    print(twitter_client.get_user_timeline_tweets(1))

#    twitter_streamer = TwitterStreamer()
#    twitter_streamer.stream_tweets(fetched_tweets_filename, hash_tag_list)